Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT Fundamentals**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy.fft import fft, ifft
#from scipy.fft import fft, ifft

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT Fundamentals**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de# Discrete Fourier Transform (DFT)

## Input Signal

Let us first define a **complex-valued signal** $x[k]$ of a certain block length $N$ ranging from $0\leq k\leq N-1$.

The variable `tmpmu` defines the frequency of the signal. We will see later how this is connected to the DFT.
For now on, leave it with `tmpmu=1`. This results in exactly one period of cosine and sine building the complex signal. If `tmpmu=2` we get exactly two periods of cos/sin. We'll get get an idea of `tmpmu`...

In [ ]:
N = 2**3  # signal block length
k = np.arange(N)  # all required sample/time indices
A = 10  # signal amplitude

tmpmu = 2-1/2  # DFT eigenfrequency worst case
tmpmu = 1  # DFT eigenfrequency best case

x = A * np.exp(tmpmu * +1j*2*np.pi/N * k)

# plot
plt.stem(k, np.real(x), markerfmt='C0o',
         basefmt='C0:', linefmt='C0:', label='real')
plt.stem(k, np.imag(x), markerfmt='C1o',
         basefmt='C1:', linefmt='C1:', label='imag')
# note that connecting the samples by lines is actually wrong, we
# use it anyway for more visual convenience:
plt.plot(k, np.real(x), 'C0-', lw=0.5)
plt.plot(k, np.imag(x), 'C1-', lw=0.5)
plt.xlabel(r'sample $k$')
plt.ylabel(r'complex-valued input signal $x[k]$')
plt.legend()
plt.grid(True)

We will now perform an DFT of $x[k]$ since we are interested in the frequency spectrum of it.

## DFT Definition

The discrete Fourier transform pair for a discrete-time signal $x[k]$ with sample index $k$ and the corresponding DFT spectrum $X[\mu]$ with frequency index $\mu$ is given as 
\begin{align}
\text{DFT}: X[\mu]=&\sum_{k=0}^{N-1}x[k]\cdot\mathrm{e}^{-\mathrm{j}\frac{2\pi}{N}k\mu}\\
\text{IDFT}: x[k]=\frac{1}{N}&\sum_{\mu=0}^{N-1}X[\mu]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}
\end{align}

Note the sign reversal in the exp()-function and the $1/N$ normalization in the IDFT. This convention is used by the majority of DSP text books and also in Python's `numpy.fft.fft()`, `numpy.fft.ifft()` and Matlab's `fft()`, `ifft()` routines.

## DFT and IDFT with For-Loops

We are now going to implement the DFT and IDFT with for-loop handling. While this might be helpful to validate  algorithms in its initial development phase, this should be avoided for practical used code in the field: for-loops are typically slow and very often more complicated to read than appropriate set up matrices and vectors. Especially for very large $N$ the computation time is very long.

Anyway, the for-loop concept is: the DFT can be implemented with an outer for loop iterating over $\mu$ and an inner for loop summing over all $k$ for a specific $\mu$.

We use variable with _ subscript here, in order to save nice variable names for the matrix based calculation.

In [ ]:
# DFT with for-loop:
X_ = np.zeros((N, 1), dtype=complex)  # alloc RAM, init with zeros
for mu_ in range(N):  # do for all DFT frequency indices
    for k_ in range(N):  # do for all sample indices
        X_[mu_] += x[k_] * np.exp(-1j*2*np.pi/N*k_*mu_)

IDFT with outer and inner looping reads as follows.

In [ ]:
# IDFT with for-loop:
x_ = np.zeros((N, 1), dtype=complex)  # alloc RAM, init with zeros
for k_ in range(N):
    for mu_ in range(N):
        x_[k_] += X_[mu_] * np.exp(+1j*2*np.pi/N*k_*mu_)
x_ *= 1/N  # normalization in the IDFT stage

Besides exchanged variables, main differences are sign reversal in exp() and the $1/N$ normalization. This is expected due to the DFT/IDFT equation pair given above.

## DFT and IDFT with Matrix Multiplication

Now we do a little better: We should think of the DFT/IDFT in terms of a matrix operation setting up a set of linear equations.

For that we define a column vector containing the samples of the discrete-time signal $x[k]$
\begin{equation}
\mathbf{x}_k = (x[k=0], x[k=1], x[k=2], \dots , x[k=N-1])^\mathrm{T}
\end{equation}

and a column vector containing the DFT coefficients $X[\mu]$

\begin{equation}
\mathbf{x}_\mu = (X[\mu=0], X[\mu=1], X[\mu=2], \dots, X[\mu=N-1])^\mathrm{T}
\end{equation}

Then, the matrix operations

\begin{align}
\text{DFT:   } & \mathbf{x}_\mu = \mathbf{W}^* \mathbf{x}_k\\
\text{IDFT:   } & \mathbf{x}_k = \frac{1}{N} \mathbf{W} \mathbf{x}_\mu
\end{align}

hold.

$()^\mathrm{T}$ is the transpose, $()^*$ is the conjugate complex.


The $N\times N$ Fourier matrix is defined as (element-wise operation $\odot$)
\begin{equation}
\mathbf{W} = \mathrm{e}^{+\mathrm{j}\frac{2\pi}{N} \odot \mathbf{K}}
\end{equation}
using the so called twiddle factor (note that the sign in the exp() is our convention)
\begin{equation}
W_N = \mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}}
\end{equation}
and the outer product
\begin{equation}
\mathbf{K} = 
\begin{bmatrix}
0\\
1\\
2\\
\vdots\\
N-1
\end{bmatrix}
\cdot
\begin{bmatrix}
0 & 1 & 2 & \cdots & N-1
\end{bmatrix}
\end{equation}
containing all possible products $k\,\mu$ in a suitable arrangement.

For the simple case $N=4$ these matrices are
\begin{align}
\mathbf{K} = \begin{bmatrix}
0 & 0 & 0 & 0\\
0 & 1 & 2 & 3\\
0 & 2 & 4 & 6\\
0 & 3 & 6 & 9
\end{bmatrix}
\rightarrow
\mathbf{W} = \begin{bmatrix}
1 & 1 & 1 & 1\\
1 & +\mathrm{j} & -1 & -\mathrm{j}\\
1 & -1 & 1 & -1\\
1 & -\mathrm{j} & -1 & +\mathrm{j}
\end{bmatrix}
\end{align}

In [ ]:
# k = np.arange(N)  # all required sample/time indices, already defined above

# all required DFT frequency indices, actually same entries like in k
mu = np.arange(N)

# set up matrices
K = np.outer(k, mu)  # get all possible entries k*mu in meaningful arrangement
W = np.exp(+1j * 2*np.pi/N * K)  # analysis matrix for DFT

In [ ]:
# visualize the content of the Fourier matrix
# we've already set up (use other N if desired):
# N = 8
# k = np.arange(N)
# mu = np.arange(N)
# W = np.exp(+1j*2*np.pi/N*np.outer(k, mu))  # set up Fourier matrix

fig, ax = plt.subplots(1, N)
fig.set_size_inches(6, 6)
fig.suptitle(
    r'Fourier Matrix for $N=$%d, blue: $\Re(\mathrm{e}^{+\mathrm{j} \frac{2\pi}{N} \mu k})$, orange: $\Im(\mathrm{e}^{+\mathrm{j} \frac{2\pi}{N} \mu k})$' % N)

for tmp in range(N):
    ax[tmp].set_facecolor('lavender')
    ax[tmp].plot(W[:, tmp].real, k, 'C0o-', ms=7, lw=0.5)
    ax[tmp].plot(W[:, tmp].imag, k, 'C1o-.', ms=7, lw=0.5)
    ax[tmp].set_ylim(N-1, 0)
    ax[tmp].set_xlim(-5/4, +5/4)
    if tmp == 0:
        ax[tmp].set_yticks(np.arange(0, N))
        ax[tmp].set_xticks(np.arange(-1, 1+1, 1))
        ax[tmp].set_ylabel(r'$\longleftarrow k$')
    else:
        ax[tmp].set_yticks([], minor=False)
        ax[tmp].set_xticks([], minor=False)
    ax[tmp].set_title(r'$\mu=$%d' % tmp)
fig.tight_layout()
fig.subplots_adjust(top=0.91)

fig.savefig('fourier_matrix.png', dpi=300)

# TBD: row version for analysis

## Fourier Matrix Properties

The DFT and IDFT basically solve two sets of linear equations, that are linked as forward and inverse problem.

This is revealed with the important property of the Fourier matrix

\begin{equation}
\mathbf{W}^{-1}
= \frac{\mathbf{W}^\mathrm{H}}{N}
= \frac{\mathbf{W}^\mathrm{*}}{N},
\end{equation}

the latter holds since the matrix is symmetric.

Thus, we see that by our convention, the DFT is the inverse problem (signal analysis) and the IDFT is the forward problem (signal synthesis)

\begin{align}
\text{DFT:   } & \mathbf{x}_\mu = \mathbf{W}^* \mathbf{x}_k \rightarrow \mathbf{x}_\mu = N \mathbf{W}^{-1} \, \mathbf{x}_k\\
\text{IDFT:   } & \mathbf{x}_k = \frac{1}{N} \mathbf{W} \mathbf{x}_\mu.
\end{align}

The occurrence of the $N$, $1/N$ factor is due to the prevailing convention in signal processing literature.

If the matrix is normalised as $\frac{\mathbf{W}}{\sqrt{N}}$, a so called unitary matrix results, for which the 
important property
\begin{equation}
(\frac{\mathbf{W}}{\sqrt{N}})^\mathrm{H} \, (\frac{\mathbf{W}}{\sqrt{N}}) = \mathbf{I} =
(\frac{\mathbf{W}}{\sqrt{N}})^{-1} \, (\frac{\mathbf{W}}{\sqrt{N}})
\end{equation}
holds, i.e. the complex-conjugate, transpose is equal to the inverse
$(\frac{\mathbf{W}}{\sqrt{N}})^\mathrm{H} = (\frac{\mathbf{W}}{\sqrt{N}})^{-1}$
and due to the matrix symmetry also
$(\frac{\mathbf{W}}{\sqrt{N}})^* =
(\frac{\mathbf{W}}{\sqrt{N}})^{-1}$
is valid.

This tells that the matrix $\frac{\mathbf{W}}{\sqrt{N}}$ is **orthonormal**, i.e. the matrix spans a orthonormal vector basis (the best what we can get in linear algebra world to work with) of $N$ normalized DFT eigensignals.

So, DFT and IDFT is transforming vectors into other vectors using the vector basis of the Fourier matrix.


## Check DFT Eigensignals and -Frequencies

The columns of the Fourier matrix $\mathbf{W}$ contain the eigensignals of the DFT. These are
\begin{align}
w_\mu[k] = \cos(\frac{2\pi}{N} k \mu) + \mathrm{j} \sin(\frac{2\pi}{N} k \mu)
\end{align}
since we have intentionally set up the matrix this way.

The plot below shows the eigensignal for $\mu=1$, which fits again one signal period in the block length $N$.
For $\mu=2$ we obtain two periods in one block.

The eigensignals for $0\leq \mu \leq N-1$ therefore exhibit a certain digital frequency, the so called DFT eigenfrequencies.

What eigensignal corresponds to $\mu=0$?...

In [ ]:
tmpmu = 1  # column index

plt.stem(k, np.real(W[:, tmpmu]), label='real',
         markerfmt='C0o', basefmt='C0:', linefmt='C0:')
plt.stem(k, np.imag(W[:, tmpmu]), label='imag',
         markerfmt='C1o', basefmt='C1:', linefmt='C1:')
# note that connecting the samples by lines is actually wrong, we
# use it anyway for more visual convenience
plt.plot(k, np.real(W[:, tmpmu]), 'C0-', lw=0.5)
plt.plot(k, np.imag(W[:, tmpmu]), 'C1-', lw=0.5)
plt.xlabel(r'sample $k$')
plt.ylabel(r'DFT eigensignal = '+str(tmpmu+1)+'. column of $\mathbf{W}$')
plt.legend()
plt.grid(True)

The nice thing about the chosen eigenfrequencies, is that the eigensignals are **orthogonal**.

This choice of the vector basis is on purpose and one of the most important ones in linear algebra and signal processing.

We might for example check orthogonality with the **complex** inner product of some matrix columns.

In [ ]:
np.dot(np.conj(W[:, 0]), W[:, 0])  # same eigensignal, same eigenfrequency
# np.vdot(W[:,0],W[:,0])  # this is the suitable numpy function

In [ ]:
np.dot(np.conj(W[:, 0]), W[:, 1])  # different eigensignals
# np.vdot(W[:,0],W[:,1])  # this is the suitable numpy function
# result should be zero, with numerical precision close to zero:

## Initial Example: IDFT Signal Synthesis for N=8

Let us synthesize a discrete-time signal by using the IDFT in matrix notation for $N=8$.

The signal should contain a DC value, the first and second eigenfrequency with different amplitudes, such as

\begin{equation}
\mathbf{x}_\mu = [8, 2, 4, 0, 0, 0, 0, 0]^\text{T}
\end{equation}

using large `X_test` in code.

In [ ]:
if N == 8:
    X_test = np.array([8, 2, 4, 0, 0, 0, 0, 0])
    # x_test = 1/N*W@X_test  # >= Python3.5
    x_test = 1/N * np.matmul(W, X_test)

    plt.stem(k, np.real(x_test), label='real',
             markerfmt='C0o', basefmt='C0:', linefmt='C0:')
    plt.stem(k, np.imag(x_test), label='imag',
             markerfmt='C1o', basefmt='C1:', linefmt='C1:')    
    # note that connecting the samples by lines is actually wrong, we
    # use it anyway for more visual convenience
    plt.plot(k, np.real(x_test), 'C0o-', lw=0.5)
    plt.plot(k, np.imag(x_test), 'C1o-', lw=0.5)
    plt.xlabel(r'sample $k$')
    plt.ylabel(r'$x[k]$')
    plt.legend()
    plt.grid(True)

    # check if results are identical with numpy ifft package
    print(np.allclose(ifft(X_test), x_test))
    print('DC is 1 as expected: ', np.mean(x_test))

This is a linear combination of the Fourier matrix columns, which are the DFT eigensignals, as 

In [ ]:
if N == 8:
    x_test2 = X_test[0] * W[:, 0] + X_test[1] * W[:, 1] + X_test[2] * W[:, 2]

We don't need summing the other columns, since their DFT coefficients in `X_test` are zero.

Finally, normalizing yields the IDFT.

In [ ]:
if N == 8:
    x_test2 *= 1/N
    print(np.allclose(x_test, x_test2))  # check with result before

## Initial Example: DFT Spectrum Analysis for N=8

Now, let us calculate the DFT of the signal `x_test`. As result, we'd expect the DFT vector

\begin{equation}
\mathbf{x}_\mu = [8, 2, 4, 0, 0, 0, 0, 0]^\text{T}
\end{equation}

that we started from.

In [ ]:
if N == 8:
    # X_test2 = np.conj(W)@x_test  # >= Python3.5
    X_test2 = np.matmul(np.conj(W), x_test)  # DFT, i.e. analysis
    print(np.allclose(X_test, X_test2))  # check with result before

This looks good. It is advisable also to check against the `numpy.fft` implementation:

In [ ]:
if N == 8:
    print(np.allclose(fft(x_test), X_test))

Besides different quantization errors in range $10^{-15...-16}$ (which is prominent even with 64Bit double precision calculation)
all results produce the same output.

The analysis stage for the discrete-time signal domain, i.e. the DFT
can be reinvented by some intuition:
How 'much' of the reference signal $\mathbf{w}_{\text{column i}}$
(any column in $\mathbf{W}$)
is contained in the discrete-time signal $\mathbf{x}_k$ that is to be analysed.

In signal processing / statistic terms we look for the amount of correlation
of the signals
$\mathbf{w}_{\text{column i}}$ and $\mathbf{x}_k$.

In linear algebra terms we are interested in the projection of $\mathbf{x}_k$ onto
$\mathbf{w}_{\text{column i}}$, because the resulting length of this vector
reveals the amount of correlation, which is precisely one DFT coefficient $X[\cdot]$.

The complex inner products $\mathbf{w}_{\text{column i}}^\text{H} \cdot \mathbf{x}_k$
reveals these searched quantities.

In [ ]:
if N == 8:
    print(np.conj(W[:, 0])@x_test)
    print(np.conj(W[:, 1])@x_test)
    print(np.conj(W[:, 2])@x_test)

Doing this for all columns of matrix $\mathbf{W}$, all DFT coefficients are obtained, such as

\begin{align}
X[\mu=0] =& \mathbf{w}_{\text{column 1}}^\text{H} \cdot \mathbf{x}_k\\
X[\mu=1] =& \mathbf{w}_{\text{column 2}}^\text{H} \cdot \mathbf{x}_k\\
X[\mu=2] =& \mathbf{w}_{\text{column 3}}^\text{H} \cdot \mathbf{x}_k\\
X[\mu=3] =& \mathbf{w}_{\text{column 4}}^\text{H} \cdot \mathbf{x}_k\\
&\vdots\\
X[\mu=N-1] =& \mathbf{w}_{\text{column N}}^\text{H} \cdot \mathbf{x}_k.
\end{align}

Naturally, all operations can be merged to one single
matrix multiplication using the conjugate transpose of $\mathbf{W}$.

\begin{equation}
\mathbf{x}_\mu = \mathbf{W}^\text{H} \cdot \mathbf{x}_k = \mathbf{W}^* \cdot \mathbf{x}_k
\end{equation}

That's what we have performed with the single liner `X_test2 = np.matmul(np.conj(W), x_test)`

## Example: Plot the DFT Magnitude Spectrum

We should now be familiar with the DFT and IDFT basic idea.

Now, let us **return to our initially created signal** `x` at the very beginning of this notebook. We want to explore and learn to interpret the DFT magnitude spectrum of it. So, we'd perform a DFT first.

In [ ]:
X = fft(x)
# print(np.allclose(np.conj(W)@x, X))  # >=Python 3.5
print(np.allclose(np.matmul(np.conj(W), x), X))

Next, let us plot the magnitude of the spectrum over $\mu$.

- We should play around with the variable `tmpmu` when defining the input signal at the very beginning of the notebook. For example we can check what happens for `tmpmu = 1`, `tmpmu = 2` and run the whole notebook to visualize the actual magnitude spectra.

We should recognize the link of the 'energy' at $\mu$ in the magnitude spectrum with the chosen `tmpmu`.

- We can apply any real valued `tmpmu` for creating the input signal, for example
    - `tmpmu = N+1`, `tmpmu = N+2`
    - `tmpmu = 1.5`
    
We should explain what happens in these cases. Recall periodicity and eigenfrequencies/-signals as fundamental concepts.

In [ ]:
plt.stem(mu, np.abs(X)/N, markerfmt='C0o', basefmt='C0:', linefmt='C0:')
# plt.plot(mu, np.abs(X)/N, 'C0', lw=1)  # this is here a misleading plot and hence not used
plt.xlabel(r'DFT eigenfrequency $\mu$')
plt.ylabel(r'DFT spectrum magnitude $\frac{|X[\mu]|}{N}$')
plt.grid(True)

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT Fundamentals**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft
#from scipy.fft import fft, ifft
from scipy.special import diric


def dft2dtft(X, Om):  # from dft_to_dtft_interpolation.ipynb
    N = len(X)
    Xi = np.zeros(len(Om), dtype='complex')
    for i, Omega in enumerate(Om):
        for mu_dft in range(N):
            Xi[i] += X[mu_dft] *\
                np.exp(-1j/2*(Omega - 2*np.pi/N*mu_dft)*(N-1)) *\
                diric(Omega - 2*np.pi/N*mu_dft, N)
    return Xi


width = 10
height = 10/16 * width
figsize = (width, height)

# Calculate DFT / DTFT

In [ ]:
# DFT stuff
N = 16  # DFT block size

k = np.arange(N)  # time index
mu = np.arange(N)  # frequency index for DFT
dOm = 2*np.pi / N  # DFT's Omega resolution
twiddle = np.exp(+1j*dOm)  # basis twiddle factor
# 1/np.sqrt(N) yields an orthonormal!!! Fourier matrix:
W = (twiddle**np.outer(k, k)) / np.sqrt(N)

# create signal

# frequency scale factor in term sof DFT's dOm:
mu_x = 1.0  # best case = is a DFT eigensignal
# mu_x = 1.5  # worst case -> between two DFT eigensignals

tmp = np.exp(+1j*dOm*k * mu_x) / np.sqrt(N)
x = tmp  # complex signal
# x = tmp.real  # real signal-> cosine
# x = tmp.imag  # real signal-> sine

# calc DFT
X = W.conj() @ x
# X_tmp = fft(x) * 1/np.sqrt(N)  # needs 1/np.sqrt(N) to be consistent with above defined W!!!
# print(np.allclose(X, X_tmp))

# calc IDFT
x_tmp = W @ X
print('x = IDFT(DFT(x)): ', np.allclose(x, x_tmp))
# x_tmp = ifft(X) * np.sqrt(N)  # needs 1/np.sqrt(N) to be consistent with above defined W!!!
# print(np.allclose(x, x_tmp))

# calc DFT -> DTFT interpolation
N_dtft = 2**10  # number of frequencies along unit circle at which DTFT values are calc
Om_dtft = np.arange(N_dtft) * 2*np.pi/N_dtft  # set up frequency vector
X_dtft = dft2dtft(X, Om_dtft)  # perform interp

# Signal Model Corresponding to DFT

- both, the signal and the DFT spectrum are periodic in $N$
- both, the signal and the DFT spectrum are discrete signals

## Plot Real/Imaginary Parts of Signal and DFT Spectrum

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
ax[0, 0].stem(k, x.real, basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 0].stem(k, x.imag, basefmt='C1:', linefmt='C1:', markerfmt='C1o')
ax[0, 1].stem(mu, X.real, basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 1].stem(mu, X.imag, basefmt='C1:', linefmt='C1:', markerfmt='C1o')
for i in range(2):
    ax[i, 0].set_xticks(np.arange(N))
    ax[i, 1].set_xticks(np.arange(N))
    ax[i, 0].set_ylim(-1/np.sqrt(N), +1/np.sqrt(N))
    ax[i, 1].set_ylim(-1, +1)
    ax[i, 0].grid(True, color='lightgray')
    ax[i, 1].grid(True, color='lightgray')
ax[0, 0].set_title(r'periodic signal,  $\Re\{x[k]\}$')
ax[1, 0].set_title(r'$\Im\{x[k]\}$')
ax[0, 1].set_title(r'periodic, discrete DFT spectrum, $\Re\{X[\mu]\}$')
ax[1, 1].set_title(r'$\Im\{X[\mu]\}$')
ax[1, 0].set_xlabel(r'$k$')
ax[1, 1].set_xlabel(r'$\mu$');

## Plot Real/Imaginary Parts of Signal and Magnitude/Phase of DFT Spectrum

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
ax[0, 0].stem(k, x.real, basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 0].stem(k, x.imag, basefmt='C1:', linefmt='C1:', markerfmt='C1o')
ax[0, 1].stem(mu, np.abs(X), basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 1].stem(mu, np.angle(X), basefmt='C1:', linefmt='C1:', markerfmt='C1o')
for i in range(2):
    ax[i, 0].set_xticks(np.arange(N))
    ax[i, 1].set_xticks(np.arange(N))
    ax[i, 0].set_ylim(-1/np.sqrt(N), +1/np.sqrt(N))
    ax[i, 0].grid(True, color='lightgray')
    ax[i, 1].grid(True, color='lightgray')
ax[0, 1].set_ylim(-1, +1)
ax[1, 1].set_ylim(-np.pi, +np.pi)
ax[0, 0].set_title(r'periodic signal,  $\Re\{x[k]\}$')
ax[1, 0].set_title(r'$\Im\{x[k]\}$')
ax[0, 1].set_title(r'periodic, discrete DFT spectrum, $|X[\mu]|$')
ax[1, 1].set_title(r'$\arg(X[\mu])$ in rad')
ax[1, 0].set_xlabel(r'$k$')
ax[1, 1].set_xlabel(r'$\mu$');

# Signal Model Corresponding to DTFT

- the signal $x[k]$ is zero for $k<0$ and $k>N-1$, thus **non-periodic**
- the signal is discrete
- the DTFT spectrum is periodic in $2\pi$
- the DTFT spectrum is continous

## Plot Real/Imaginary Parts of Signal and DTFT Spectrum

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
ax[0, 0].stem(k, x.real, basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 0].stem(k, x.imag, basefmt='C1:', linefmt='C1:', markerfmt='C1o')
ax[0, 1].stem((mu*dOm)/dOm, X.real, basefmt='C0:', linefmt='C0:', markerfmt='C4.')
ax[1, 1].stem((mu*dOm)/dOm, X.imag, basefmt='C1:', linefmt='C1:', markerfmt='C3.')
ax[0, 1].plot(Om_dtft/dOm, X_dtft.real, 'C0')
ax[1, 1].plot(Om_dtft/dOm, X_dtft.imag, 'C1')
for i in range(2):
    ax[i, 1].set_xlim((0, N))
    ax[i, 0].set_xticks(np.arange(N))
    ax[i, 1].set_xticks(np.arange(N))
    ax[i, 0].set_ylim(-1/np.sqrt(N), +1/np.sqrt(N))
    ax[i, 1].set_ylim(-1, +1)
    ax[i, 0].grid(True, color='lightgray')
    ax[i, 1].grid(True, color='lightgray')
ax[0, 0].set_title(r'non-periodic signal (zero outside), $\Re\{x[k]\}$')
ax[1, 0].set_title(r'$\Im\{x[k]\}$')
ax[0, 1].set_title(r'periodic, continuous DTFT spectrum, $\Re\{X(\Omega)\}$')
ax[1, 1].set_title(r'$\Im\{X(\Omega)\}$')
ax[1, 0].set_xlabel(r'$k$')
ax[1, 1].set_xlabel(r'$\Omega / (\frac{2 \pi}{N})$')
ax[0, 1].text(1, -0.7, 'dots represent the DFT coefficients');

## Plot Real/Imaginary Parts of Signal and Magnitude/Phase of DTFT Spectrum

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
ax[0, 0].stem(k, x.real, basefmt='C0:', linefmt='C0:', markerfmt='C0o')
ax[1, 0].stem(k, x.imag, basefmt='C1:', linefmt='C1:', markerfmt='C1o')
ax[0, 1].stem((mu*dOm)/dOm, np.abs(X), basefmt='C0:', linefmt='C0:', markerfmt='C4.')
ax[1, 1].stem((mu*dOm)/dOm, np.angle(X), basefmt='C1:', linefmt='C1:', markerfmt='C3.')
ax[0, 1].plot(Om_dtft/dOm, np.abs(X_dtft), 'C0')
ax[1, 1].plot(Om_dtft/dOm, np.angle(X_dtft), 'C1')
for i in range(2):
    ax[i, 1].set_xlim((0, N))
    ax[i, 0].set_xticks(np.arange(N))
    ax[i, 1].set_xticks(np.arange(N))
    ax[i, 0].set_ylim(-1/np.sqrt(N), +1/np.sqrt(N))
    ax[i, 0].grid(True, color='lightgray')
    ax[i, 1].grid(True, color='lightgray')
ax[0, 1].set_ylim(-1, +1)
ax[1, 1].set_ylim(-np.pi, +np.pi)
ax[0, 0].set_title(r'non-periodic signal (zero outside), $\Re\{x[k]\}$')
ax[1, 0].set_title(r'$\Im\{x[k]\}$')
ax[0, 1].set_title(r'periodic, continuous DTFT spectrum, $|X(\Omega)|$')
ax[1, 1].set_title(r'$arg(X(\Omega))$ in rad')
ax[1, 0].set_xlabel(r'$k$')
ax[1, 1].set_xlabel(r'$\Omega / (\frac{2 \pi}{N})$')
ax[0, 1].text(1, -0.7, 'dots represent the DFT coefficients');

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT to DTFT Interpolation with the Periodic Sinc Function**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from numpy.fft import fft, ifft
#from scipy.fft import fft, ifft
from scipy.special import diric

# Sampling of DTFT towards DFT 

We should create a signal and the corresponding DTFT spectrum for which we have analytical expressions.

We can use the exact Hamming window from the tutorial (see Ch. 1.8.4) found at https://github.com/spatialaudio/digital-signal-processing-exercises/blob/master/dft/dft_windowing_tutorial/dft_windowing_tutorial.pdf

In [ ]:
N_DTFT = 2**3
beta = 21/25  # exact Hamming coeff, see
k_DTFT = np.arange(N_DTFT)
x_DTFT = 1 - beta * np.cos(2*np.pi/N_DTFT*(k_DTFT+1/2))
Omega_DTFT = np.arange(-4*np.pi, +4*np.pi, np.pi/2**6)
X_DTFT = np.exp(-1j*Omega_DTFT*(N_DTFT-1)/2)*(N_DTFT*diric(Omega_DTFT, N_DTFT) + beta/2 *
                                              N_DTFT*diric(Omega_DTFT-2*np.pi/N_DTFT, N_DTFT) + beta/2 *
                                              N_DTFT*diric(Omega_DTFT+2*np.pi/N_DTFT, N_DTFT))

We can zeropad the signal before the DFT. We do this manually, to make sure that we are aware of this concept.

In [ ]:
# how many samples should be used for zeropadding of the x_DTFT signal
# if 0 then the signal has length of N_DTFT, so N_DTFT==N_DFT
Nzeropad = 5  

x_DFT = np.concatenate([x_DTFT, np.zeros(Nzeropad)])

Then we perform the DFT and calculate at which frequencies $\Omega_\mu$ the DFT coefficients occur:

In [ ]:
# calculate the DFT of the zeropadded signal
N_DFT = x_DFT.size
#print('DFT size check must be zero:', N_DFT - N_DTFT - Nzeropad)
print('DFT block size:', N_DFT)
k_DFT = np.arange(N_DFT)
X_DFT = fft(x_DFT)
Omega_DFT = np.arange(N_DFT) * 2*np.pi/N_DFT

By plotting the spectrum of the DTFT and the DFT, we see that the DFT coefficients exactly correspond to DTFT values at specific frequencies. This is due to the inherent sampling process, where we **sample the DTFT spectrum** to **precisely obtain the DFT coefficients**. Detailed derivation can be found in the [DFT tutorial in Ch. 1.7](https://github.com/spatialaudio/digital-signal-processing-exercises/blob/master/dft/dft_windowing_tutorial/dft_windowing_tutorial.pdf) 

**Sampling in one domain** (here the spectrum) corresponds to **'make it periodic' in the other domain** (here the discrete-time signal). The single window function that corresponds to the DTFT spectrum (blue) is thus repeated over and over again (orange), which is required for the DFT's periodicity characteristic. Its periodicity is determined by the DFT size and not necessarily equal to `N_DTFT`.

Very **large zeropadding** can be used to obtain many spectral samples and by that a very good **approximation of the DTFT** spectrum **without** using **explicit interpolation** between the DFT coefficients. This is often used in practice, very conveniently by calling `fft(x, N)` where `N` is larger than length of `x`. Note however, that by doing so, we do not increase the spectral resolution and we also do not enhance the capability for spectral discrimination. We gain **no new information by oversampling**.

Note that `N_DTFT=N_DFT` (no zeropadding) is the case of **critical sampling**.

In [ ]:
plt.figure(figsize=(10, 16))
ax = plt.subplot(5, 1, 1)
plt.plot(Omega_DTFT/np.pi, np.real(X_DTFT), label='DTFT')
plt.plot(Omega_DFT/np.pi, np.real(X_DFT), 'C3o', ms=3, label='DFT')
plt.plot((Omega_DFT-2*np.pi)/np.pi, np.real(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT-4*np.pi)/np.pi, np.real(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT+2*np.pi)/np.pi, np.real(X_DFT), 'C3o', ms=3)
plt.xlim(-4, 4)
plt.xlabel(r'$\Omega / \pi$')
plt.ylabel(r'spectrum, real part')
plt.legend(loc='upper left')
rect = plt.Rectangle((0, -3.5), 2, 12, facecolor='lightgrey')
ax.add_patch(rect)
plt.text(0.5, 6, '1st period')

plt.subplot(5, 1, 2)
ax = plt.plot(Omega_DTFT/np.pi, np.imag(X_DTFT))
plt.plot(Omega_DFT/np.pi, np.imag(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT-2*np.pi)/np.pi, np.imag(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT-4*np.pi)/np.pi, np.imag(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT+2*np.pi)/np.pi, np.imag(X_DFT), 'C3o', ms=3)
plt.xlim(-4, 4)
plt.xlabel(r'$\Omega / \pi$')
plt.ylabel(r'spectrum, imaginary part')

plt.subplot(5, 1, 3)
ax = plt.plot(Omega_DTFT/np.pi, np.abs(X_DTFT))
plt.plot(Omega_DFT/np.pi, np.abs(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT-2*np.pi)/np.pi, np.abs(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT-4*np.pi)/np.pi, np.abs(X_DFT), 'C3o', ms=3)
plt.plot((Omega_DFT+2*np.pi)/np.pi, np.abs(X_DFT), 'C3o', ms=3)
plt.xlim(-4, 4)
plt.xlabel(r'$\Omega / \pi$')
plt.ylabel(r'spectrum, magnitude')

plt.subplot(5, 1, 4)
plt.stem(k_DTFT, x_DTFT, basefmt='C0:', linefmt='C0:', markerfmt='C0o',
         label='DTFT (non-periodic, length = '+str(N_DTFT)+' samples')
plt.stem(k_DTFT-N_DTFT, x_DTFT*0, basefmt='C0:',
         linefmt='C0:', markerfmt='C0o')
plt.stem(k_DTFT+N_DTFT, x_DTFT*0, basefmt='C0:',
         linefmt='C0:', markerfmt='C0o')
plt.xlabel(r'$k$')
plt.ylabel(r'$x_{DTFT}[k]$')
plt.legend(loc='upper left')

plt.subplot(5, 1, 5)
plt.stem(k_DFT, x_DFT, basefmt='C3:', linefmt='C3:', markerfmt='C3o',
         label='DFT (periodicity = '+str(N_DFT)+' samples')
plt.stem(k_DFT-N_DFT, x_DFT, basefmt='C3:', linefmt='C3:', markerfmt='C3o')
plt.stem(k_DFT+N_DFT, x_DFT, basefmt='C3:', linefmt='C3:', markerfmt='C3o')
plt.xlabel(r'$k$')
plt.ylabel(r'$x_{DFT}[k]$')
plt.legend(loc='upper left');

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT to DTFT Interpolation with the Periodic Sinc Function**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from numpy.fft import fft, ifft
#from scipy.fft import fft, ifft
from scipy.special import diric

# DFT to DTFT Interpolation with the Periodic Sinc Function

The DFT spectrum $X[\mu]$ can be interpolated towards the DTFT spectrum $X(\Omega)$ with

\begin{equation}
X(\Omega)=\sum_{\mu=0}^{N-1}X[\mu]\cdot\mathrm{e}^{-\mathrm{j}\frac{\left(\Omega-\frac{2\pi}{N}\mu\right)(N-1)}{2}}\cdot\text{psinc}_N\left(\Omega-\frac{2\pi}{N}\mu\right).
\end{equation}

This interpolation implies:
- the DFT $X[\mu]$ stems from a signal $x[k]$ for which periodicity of $N$ is inherent, we assume the first period at $0\leq k\leq N-1$ 
- the DFT spectrum is discrete and $N$ periodic
- the DTFT spectrum stems from the signal $x[k]$ for $0\leq k\leq N-1$, for all other $k$ is $x[k]=0$, i.e. considering only one period
- the DTFT spectrum is continuous and $2\pi$ periodic, which means that the signal $x[k]$ can not be periodic

The interpolation kernel utilises the so-called **periodic sinc function**

\begin{align}
\text{psinc}_N(\Omega)=\begin{cases}\frac{1}{N}\cdot\frac{\sin\left(\frac{N}{2}\Omega\right)}{\sin\left(\frac{1}{2}\Omega\right)}&\text{for }\Omega\neq2\pi m\\
(-1)^{m(N-1)}&\text{for }\Omega=2\pi m\end{cases},\,\,m\in\mathbb{Z},
\end{align}

which is also known as aliased sinc and Dirichlet function.

Below, we give an example graph for $\text{psinc}_N(\Omega)$.
Note that the orange dots indicate the DFT eigenfrequencies.

In [ ]:
def dft2dtft(X, W):
    N = len(X)
    Xi = np.zeros(len(W), dtype='complex')
    for i, Omega in enumerate(W):
        for mu_dft in range(N):
            Xi[i] += X[mu_dft] *\
                np.exp(-1j/2*(Omega - 2*np.pi/N*mu_dft)*(N-1)) *\
                diric(Omega - 2*np.pi/N*mu_dft, N)
    return Xi

In [ ]:
K = 3
N_DFT = 2**K
Omega_DFT = np.arange(-2*N_DFT, 2*N_DFT) * 2*np.pi/N_DFT
N = 2**(K+4)
Omega = np.arange(-2*N, 2*N) * 2*np.pi/N
psinc_DFT = diric(Omega_DFT, N_DFT)
psinc = diric(Omega, N_DFT)

plt.plot(Omega, psinc, label='psinc function')
plt.plot(Omega_DFT, psinc_DFT, 'o', label='DFT bins')
plt.xlim(-4*np.pi, 4*np.pi)
plt.xticks(np.arange(-4, 5, 1)*np.pi)
plt.xlabel(r'$\Omega$')
plt.ylabel(r'psinc($\Omega$)')
plt.title('psinc for '+str(N_DFT)+'-point DFT')
plt.legend()
plt.grid(True)

## Task: DFT Analysis Using a Rectangular Window

A complex signal

- $x[k]=\mathrm{exp}(\mathrm{j}(\Omega k + \frac{\pi}{\pi}))$ with
- $\Omega=4\cdot\frac{2\pi}{N}$, 
- $N=8$

is to be analysed with the DFT 
\begin{align}
X[\mu]=\sum_{k=0}^{N-1}x[k]\cdot\mathrm{e}^{-\mathrm{j}\frac{2\pi}{N}k\mu}
\end{align}
for $0\leq k \leq N-1$.

Furthermore, assume that $x[k]$ results from continuous-time signal $x(t)$ using sampling frequency of $f_s=10$ Hz.

1. Plot the discrete-time signals that correspond to the DFT and the DTFT spectrum.

2. Calculate the DFT spectrum $X[\mu]$ of $x[k]$ and visualise the real and imaginary part as well as the magnitude and the phase of $X[\mu]$ over $0\leq\mu\leq N-1$.

3. Implement the above mentioned interpolation towards the DTFT and visualise the resulting magnitude spectra $|X[\mu]|$, $|X(\Omega)|$ over frequency axes $\mu$, $\Omega$ as well as the physical frequency $f$.

## Task 1: Generate Signal and Plot

In [ ]:
N = 8

Om = 4 * 2*np.pi/N  # play with the factor 4

k = np.arange(N)
x = np.exp(1j*(Om*k+1))  # pi/pi=1

koffs = [-2, -1, 1, 2]
plt.subplot(2, 1, 1)
plt.plot(k, x.real, 'C0o-', label='real', ms=5)
plt.plot(k, x.imag, 'C1o-', label='imag', ms=5)
# the DFT spectrum corresponds to a periodic signal of period N
# we exemplarily indicate this
for koffsi in koffs:
    plt.plot(k+koffsi*N, x.real, 'C0o-', ms=3)
    plt.plot(k+koffsi*N, x.imag, 'C1o-', ms=3)
plt.xlim(koffs[0]*N, koffs[-1]*N+N-1)
plt.xlabel(r'$k$')
plt.ylabel(r'$x_\mathrm{DFT}[k]$')
plt.legend()
plt.grid(True)
plt.subplot(2, 1, 2)
plt.plot(k, x.real, 'C0o-', ms=5)
plt.plot(k, x.imag, 'C1o-', ms=5)
# the DTFT spectrum corresponds to singular event of length N for k=0...N-1
# we exemplarily indicate this
for koffsi in koffs:
    plt.plot(k+koffsi*N, 0*x.real, 'C0o-', ms=3)
    plt.plot(k+koffsi*N, 0*x.imag, 'C1o-', ms=3)
plt.xlim(koffs[0]*N, koffs[-1]*N+N-1)
plt.xlabel(r'$k$')
plt.ylabel(r'$x_\mathrm{DTFT}[k]$')
plt.grid(True)

## Task 2: DFT and Plot

In [ ]:
mu = np.arange(N)
w = np.ones(N)

# optional playground for windowing
# see dft_windowing_tutorial.pdf for the equations of these windows
# note that the scipy.signal.windows with same name behave different !
# thus standardised names does NOT imply standardised characteristics
if False:
    # Hann, two unused zeros
    w = (1 - np.cos(2*np.pi/N*(k+1/2))) / 2 
    # Hamming, put notch into first sidelobe, thus all zeros used
    w = 0.54 - 0.46 * np.cos(2*np.pi/N*(k+1/2))

X = fft(x*w)

In [ ]:
plt.figure(figsize=(8, 4))

plt.subplot(2, 2, 1)
plt.plot(mu, np.real(X), 'o')
plt.xlim(0, N-1)
plt.xlabel(r'$\mu$')
plt.ylabel(r'$\Re(X[\mu])$')
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(mu, np.imag(X), 'o')
plt.xlim(0, N-1)
plt.xlabel(r'$\mu$')
plt.ylabel(r'$\Im(X[\mu])$')
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(mu, np.abs(X), 'o')
plt.xlim(0, N-1)
plt.xlabel(r'$\mu$')
plt.ylabel(r'$|X[\mu]|$')
plt.grid(True)

# note that only the phase at mu=4 is a meaningful result (=1)
# all other values are due to numerical noise / trash
# since Re and Im are approx zero at these mu
plt.subplot(2, 2, 4)
plt.plot(mu, np.angle(X), 'o')
plt.xlim(0, N-1)
plt.xlabel(r'$\mu$')
plt.ylabel(r'$\angle X[\mu]$')
plt.grid(True)

## Task 3: DFT -> DTFT Interpolation, Plot

In [ ]:
fs = 10
Ni = 2**8
W = np.arange(Ni) * 2*np.pi/Ni  # Omega for which to interpolate to DTFT
f = W / (2*np.pi) * fs  # physical frequency based on W using fs
mui = W / (2*np.pi) * N  # frequency vector normalized to integer frequencies
Xi = dft2dtft(X, W)  # DTFT interpolation from DFT

plt.figure(figsize=(10, 10))

plt.subplot(3, 2, 1)
plt.plot(mu, np.abs(X), 'C0o', label='DFT')
plt.plot(mui, np.abs(Xi), 'C1-', label='DTFT')
plt.xlabel(r'$\mu$')
plt.ylabel('|X|')
plt.legend()
plt.grid(True)

plt.subplot(3, 2, 2)
plt.plot(mu, 20*np.log10(np.abs(X)), 'C0o', label='DFT')
plt.plot(mui, 20*np.log10(np.abs(Xi)), 'C1-', label='DTFT')
plt.ylim(-100, 20)
plt.xlabel(r'$\mu$')
plt.ylabel('20lg|X| / dB')
plt.grid(True)

plt.subplot(3, 2, 3)
plt.plot(2*np.pi/N*mu, np.abs(X), 'C0o', label='DFT')
plt.plot(W, np.abs(Xi), 'C1-', label='DTFT')
plt.xlabel(r'$\Omega$')
plt.ylabel('|X|')
plt.grid(True)

plt.subplot(3, 2, 4)
plt.plot(2*np.pi/N*mu, 20*np.log10(np.abs(X)), 'C0o', label='DFT')
plt.plot(W, 20*np.log10(np.abs(Xi)), 'C1-', label='DTFT')
plt.ylim(-100, 20)
plt.xlabel(r'$\Omega$')
plt.ylabel('20lg|X| / dB')
plt.grid(True)

plt.subplot(3, 2, 5)
plt.plot(mu*fs/N, np.abs(X), 'C0o', label='DFT')
plt.plot(f, np.abs(Xi), 'C1-', label='DTFT')
plt.xlabel(r'f / Hz')
plt.ylabel('|X|')
plt.grid(True)

plt.subplot(3, 2, 6)
plt.plot(mu*fs/N, 20*np.log10(np.abs(X)), 'C0o', label='DFT')
plt.plot(f, 20*np.log10(np.abs(Xi)), 'C1-', label='DTFT')
plt.ylim(-100, 20)
plt.xlabel(r'f / Hz')
plt.ylabel('20lg|X| / dB')
plt.grid(True)

We observe that 7 of 8 DFT coeefficients are precisely zero, and only $|X[\mu=4]|=8$. This is intentional and is expected for the chosen signal that oscillates with exactly a DFT eigenfrequency, here half of the sampling frequency.

The code above allows for two more playgrounds:

- We might change $\Omega=1\cdot\frac{2\pi}{N}$, $\Omega=2\cdot\frac{2\pi}{N}$, $\Omega=3\cdot\frac{2\pi}{N}$
and observe how the spectrum moves around the frequency axis, however the shape is preserved.
The main lobe precisely corresponds to the chosen $\mu$.

- We might change $\Omega=1.5\cdot\frac{2\pi}{N}$, $\Omega=2.5\cdot\frac{2\pi}{N}$, $\Omega=3.5\cdot\frac{2\pi}{N}$
and observe that - since these frequencies are **no** DFT eigenfrequencies - now all DFT coefficients exhibit energy. This suggests frequencies in the signal which are actually not there, but originate from cutting the signal to the chosen block size $N$. The effect is known as **leakage effect**.

- In task 2 we might optionally apply a window to the signal. The plots for task 3 then reveal how the leakage effect is reduced but the main lobe gets broader.
Since we have chosen a mono-frequent, complex oscillation as input signal to the DFT / DTFT, the actual window spectrum can be directly seen in the plots.

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT and Windowing**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft, fftshift
#from scipy.fft import fft, ifft, fftshift
from scipy.signal.windows import hann, flattop

## Task a)

Generate two sine signals of $f_1=200$ Hz and $f_2=200.25$ Hz and amplitude $|x[k]|_\text{max}=1$ for the sampling frequency $f_s=800$ Hz in the range of $0\leq k<N=1600$.

In [ ]:
# a)
f1 = 200  # Hz
f2 = 200.25  # Hz
fs = 800  # Hz
N = 1600
k = np.arange(N)
x1 = np.sin(2*np.pi*f1/fs*k)
x2 = np.sin(2*np.pi*f2/fs*k)

## Task b) 

Generate
- a rectangular window, 
- a Hann window and
- a flat top window

with the same lengths as the sine signals. Note: we analyze signals, so we use `sym=False` (periodic window) rather than `sym=True` (symmetric window, used for FIR filter design). Plot the obtained window signals over $k$.

In [ ]:
# b)
wrect = np.ones(N)
whann = hann(N, sym=False)
wflattop = flattop(N, sym=False)
plt.plot(wrect, 'C0o-', ms=3, label='rect')
plt.plot(whann, 'C1o-', ms=3, label='hann')
plt.plot(wflattop, 'C2o-', ms=3, label='flattop')
plt.xlabel(r'$k$')
plt.ylabel(r'window $w[k]$')
plt.xlim(0, N)
plt.legend()
plt.grid(True)

## Task c)

 Window both sine signals `x1` and `x2` with the three windows and calculate the corresponding DFT spectra using FFT algorithm either from `numpy.fft` or from `scipy.fft` package.

In [ ]:
# c)
X1wrect = fft(x1)
X2wrect = fft(x2)

X1whann = fft(x1*whann)
X2whann = fft(x2*whann)

X1wflattop = fft(x1*wflattop)
X2wflattop = fft(x2*wflattop)

## Task d)

Plot the **normalized** level of the DFT spectra in between 175 Hz and 225 Hz and -50 and 0 dB.

Note that we are dealing with analysis of sine signals, so a convenient **normalization** should be applied for the shown level. This can be achieved by making the result independent from the chosen DFT length $N$. Furthermore, considering negative and positive frequency bins, multiplying with 2 yields normalization to sine signal amplitudes. Since the frequency bin for 0 Hz and (if $N$ is even) for $f_s/2$ exists only once, multiplication with 2 is not required for these bins.

### Preparations for solution

It is meaningful to define a function that returns the level of DFT in term of sine signal normalization.

Furthermore, the DFT frequency vector should be set up.

In [ ]:
# this handling is working for N even and odd:
def fft2db(X):
    N = X.size
    Xtmp = 2/N * X  # independent of N, norm for sine amplitudes
    Xtmp[0] *= 1/2  # bin for f=0 Hz is existing only once, so cancel *2 from above
    if N % 2 == 0:  # fs/2 is included as a bin
        # fs/2 bin is existing only once, so cancel *2 from above
        Xtmp[N//2] = Xtmp[N//2] / 2
    return 20*np.log10(np.abs(Xtmp))  # in dB


# setup of frequency vector this way is independent of N even/odd:
df = fs/N
f = np.arange(N)*df

The proposed handling is independent of N odd/even and returns the whole DFT spectrum. Since we normalized for physical sine frequencies, only the part from 0 Hz to fs/2 is valid.
So, make sure that spectrum returned from fft2db is only plotted up to fs/2.

### Solution

In [ ]:
plt.figure(figsize=(16/1.5, 10/1.5))
plt.subplot(3, 1, 1)
plt.plot(f, fft2db(X1wrect), 'C0o-', ms=3, label='best case rect')
plt.plot(f, fft2db(X2wrect), 'C3o-', ms=3, label='worst case rect')
plt.xlim(175, 225)
plt.ylim(-60, 0)
plt.xticks(np.arange(175, 230, 5))
plt.yticks(np.arange(-60, 10, 10))
plt.legend()
#plt.xlabel('f / Hz')
plt.ylabel('A / dB')
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(f, fft2db(X1whann), 'C0o-', ms=3, label='best case hann')
plt.plot(f, fft2db(X2whann), 'C3o-', ms=3, label='worst case hann')
plt.xlim(175, 225)
plt.ylim(-60, 0)
plt.xticks(np.arange(175, 230, 5))
plt.yticks(np.arange(-60, 10, 10))
plt.legend()
#plt.xlabel('f / Hz')
plt.ylabel('A / dB')
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(f, fft2db(X1wflattop), 'C0o-', ms=3, label='best case flattop')
plt.plot(f, fft2db(X2wflattop), 'C3o-', ms=3, label='worst case flattop')
plt.xlim(175, 225)
plt.ylim(-60, 0)
plt.xticks(np.arange(175, 230, 5))
plt.yticks(np.arange(-60, 10, 10))
plt.legend()
plt.xlabel('f / Hz')
plt.ylabel('A / dB')
plt.grid(True)

## Task e)
Plot the level of the window DTFT spectra normalized to their mainlobe maximum for $-\pi \leq \Omega \leq \pi$ and -120 dB to 0 dB. Use zero-padding or the formulas for interpolation towards the DTFT to achieve a sufficiently high resolution of the spectra. To inspect the mainlobe in detail spectra might be plotted within the range $-\pi/100 \leq \Omega \leq \pi/100$ as well.

### Preparations for solution

It is again meaningful to define a function that returns the quasi-DTFT and the evaluated digital frequencies.
Here is a proposal using zeropadding (to obtain DTFT-like frequency resolution) and fftshift (to bring mainlobe into the middle of numpy array), which then requires $\Omega$ from $-\pi$ to $\pi$. 

In [ ]:
def winDTFTdB(w):
    N = w.size  # get window length
    Nz = 100*N  # zeropadding length
    W = np.zeros(Nz)  # allocate RAM
    W[0:N] = w  # insert window
    W = np.abs(fftshift(fft(W)))  # fft, fftshift and magnitude
    W /= np.max(W)  # normalize to maximum, i.e. the mainlobe maximum here
    W = 20*np.log10(W)  # get level in dB
    # get appropriate digital frequencies
    Omega = 2*np.pi/Nz*np.arange(Nz) - np.pi  # also shifted
    return Omega, W

In [ ]:
plt.plot([-np.pi, +np.pi], [-3.01, -3.01], 'gray')  # mainlobe bandwidth
plt.plot([-np.pi, +np.pi], [-13.3, -13.3], 'gray')  # rect max sidelobe
plt.plot([-np.pi, +np.pi], [-31.5, -31.5], 'gray')  # hann max sidelobe
plt.plot([-np.pi, +np.pi], [-93.6, -93.6], 'gray')  # flattop max sidelobe
Omega, W = winDTFTdB(wrect)
plt.plot(Omega, W, label='rect')
Omega, W = winDTFTdB(whann)
plt.plot(Omega, W, label='hann')
Omega, W = winDTFTdB(wflattop)
plt.plot(Omega, W, label='flattop')
plt.xlim(-np.pi, np.pi)
plt.ylim(-120, 10)

plt.xlim(-np.pi/100, np.pi/100)  # zoom into mainlobe

plt.xlabel(r'$\Omega$')
plt.ylabel(r'|W($\Omega$)| / dB')
plt.legend()
plt.grid(True)

## Task f)

Interpret the results of d) with the help of e) regarding the best and worst case for the different windows. Why do the results for the signals with frequencies $f_1$ and $f_2$ differ?

This is the key question of the windowing concept. Check the DFT/window script for possible explanations.

## Task g)

1. Determine the width of the main lobe (at the -3.01 dB cut frequencies) in terms of physical frequency and digital frequency.

2. Determine the attenuation of the highest side lobe from the window spectra.

For 1. The mainlobe bandwidth can be computed comparably easy, assumed that zeropadding resolution is high enough. Just find that Omega, for which W gets larger than -3 dB. Starting from Omega=-pi, one can not miss it. Then doubling the found value gives the bandwidth.

In [ ]:
Omega, W = winDTFTdB(wrect)
BW = Omega[W >= -3.01]
print('rect window mainlobe bandwidth is',
      (-BW[0]*2)*fs/(2*np.pi), 'Hz,', (-BW[0]*2), 'rad')

Omega, W = winDTFTdB(whann)
BW = Omega[W >= -3.01]
print('hann window mainlobe bandwidth is',
      (-BW[0]*2)*fs/(2*np.pi), 'Hz,', (-BW[0]*2), 'rad')

Omega, W = winDTFTdB(wflattop)
BW = Omega[W >= -3.01]
print('flattop window mainlobe bandwidth is',
      (-BW[0]*2)*fs/(2*np.pi), 'Hz,', (-BW[0]*2), 'rad')

For 2. This is a little bit more tricky, since you should find it starting from the mainlobe, falling into the first zero and then find the maximum level of the sidelobes. Think about how to compute this elegantly.

The plot above for task e) indicated the sidelobe levels with gray lines. You'll find that maximum sidelobe level is -13.3 dB for rectangular (always equal to first sidelobe), -31.5 dB for Hann (always equal to first sidelobe), -93.6 dB for Flattop window compared to the 0dB mainlobe maximum.

## Task h)

Explain for which signal analysis task the rectangular window and the flat top window should be used.

- Rectangular has good frequency resolution (most narrow mainlobe), but bad sidelobe suppression, thus deviations from DFT eigenfrequencies lead to large estimation errors of the spectrum's magnitude. This window exhibits the largest leakage effect but best frequency resolution.

- Flattop has bad frequency resolution (wide mainlobe), but good sidelobe suppression, thus deviations from DFT eigenfrequencies lead to small estimation errors of the spectrum's magnitude. This window has very low leakgave effect but very bad frequency resolution.

So, use a rectangular window if frequency resolution must be very high, use a flattop window if magnitude estimation error must be small.


## Task i)

Do some research on your own: Which advantages exhibit the Kaiser-Bessel and the Dolph-Chebyshev windows compared to the so far used windows here?

The Kaiser-Bessel and the Dolph-Chebyshev window are members of the so called parametric windows family.
See the Jupyter notebook [parametric_windows.ipynb](parametric_windows.ipynb) for details.
We have seen that mainlobe width and sidelobe suppression is always a trade-off. The rect, hann and flattop window are so called non-parametric windows only having window length $N$ as parameter and by that exhibit a certain, fixed mainlobe/sidelobe trade-off by design.

Parametric windows, besides length $N$ (which is often fixed due to measurement constraints), allow to change at least one more parameter, which precisely varies the trade-off between mainlobe width and sidelobe suppression. The specific trade-off characteristic follows a certain design criterion.
For example the Dolph-Chebyshev window asks for equal desired sidelobe level, whereas the Kaiser-Bessel asks for mainlobe energy concentration for a desired bandwidth. 

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**DFT/DTFT and Windowing for Superposition of 2 Complex Exponential Signals**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

# DTFT for Windowing

The signal $x[k]$ over time index $k$ is being windowed by $w[k]$, for which the corresponding **circular convolution** holds in DTFT domain

\begin{equation}
x[k] \cdot w[k] \circ-\bullet \frac{1}{2\pi} X(\Omega) \circledast_{2\pi} W(\Omega)
\end{equation}

Typically for the window function we choose non-zero values in $w[k]$ for $0 \leq k \leq N-1$ and $w[k]=0$ elsewhere.

Thus, we deal with a length $N$ signal that is non-zero at $0 \leq k \leq N-1$.
We can apply an $N$ DFT onto this signal $x_w[k] = x[0 \leq k \leq N-1] \cdot w[0 \leq k \leq N-1]$ to obtain its DFT spectrum $X_w[\mu]$ with the baseband $0\leq\mu\leq N-1$.

We are allowed to interpolate the DTFT spectrum from the DFT spectrum, (only) if we assume that $x_w[k]$ is a single occurring sequence, but is not periodic in $N$.

## Special Case: Complex Exponential

For a certain frequency $\Omega_1$ we can invent a simple complex exponential signal

\begin{equation}
x_1[k] = \mathrm{e}^{+\mathrm{j} \Omega_1 k} \circ-\bullet \bot\bot\bot (\frac{\Omega-\Omega_1}{2\pi})
\end{equation}

For certain window / DFT length $N$, the $\Omega_1 = \frac{2\pi}{N} \cdot \mu$ for $0\leq\mu\leq N-1$ correspond to the DFT eigenfrequencies in the spectrum baseband. On the other hand, if $\mu$ is rather non-integer, the frequency is between two DFT eigenfrequencies.

Since the complex exponential signal exhibits a Dirac comb in DTFT domain, the above circular convolution for the windowing process is very convenient to discuss:

The Dirac is the neutral element of the convolution.
Thus, to analyze the spectrum of $x_1[k]$ (actually its windowed part) we just need to interpret the DTFT spectrum $W(\Omega-\Omega_1)$, i.e. the window spectrum that is circular shifted along frequency axis.

## Special Case: Superposition of Two Complex Exponentials

In case of two complex exponentials

\begin{equation}
x_1[k] = \mathrm{e}^{+\mathrm{j} \Omega_1 k} \circ-\bullet \bot\bot\bot (\frac{\Omega-\Omega_1}{2\pi})
\end{equation}

\begin{equation}
x_2[k] = \mathrm{e}^{+\mathrm{j} \Omega_2 k} \circ-\bullet \bot\bot\bot (\frac{\Omega-\Omega_2}{2\pi})
\end{equation}

the superposition of these signals and windowing yields

\begin{equation}
w[k] \cdot (x_1[k] + x_2[k]) \circ-\bullet W(\Omega-\Omega_1) + W(\Omega-\Omega_2)
\end{equation}

To discuss the spectral characteristics of $x_1[k] + x_2[k]$ we need to interpret the **complex-valued** superposition
of the two spectra $W(\Omega-\Omega_1)$ and $W(\Omega-\Omega_2)$.
If we can handle this simple case, we have a better understanding for real world signal applications.

First, let's define a convenient plotting routine.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft, fftshift
#from scipy.fft import fft, ifft, fftshift
from scipy.signal.windows import kaiser


def plot_dft_dtft():  # TBD: proper scaling between DFT and DTFT !!!
    X1, X2 = fft(x1), fft(x2)
    x = x1 + x2
    X = fft(x)  # == fft(x1) + fft(x2) due to linearity

    # zeropadding to obtain a DTFT-like frequency resolution:
    Nz = 2**4 * N  # appropriate higher resolution such that curves are smooth
    muz = np.arange(Nz)
    x1z, x2z = np.zeros(Nz, dtype='complex'), np.zeros(Nz, dtype='complex')
    x1z[0:N], x2z[0:N] = x1, x2
    X1z, X2z = fft(x1z), fft(x2z)
    xz = x1z + x2z
    Xz = fft(xz)

    plt.figure(figsize=(6, 6))
    plt.subplot(2, 1, 1)
    plt.stem(mu, np.abs(X1), basefmt='C0:', linefmt='C0:', markerfmt='C0o',
             label=r'DFT $|X_1[\mu]|$')
    plt.stem(mu, np.abs(X2), basefmt='C1:', linefmt='C1:', markerfmt='C1o',
             label=r'DFT $|X_2[\mu]|$')
    plt.plot(muz / Nz * N, np.abs(X1z), 'C0',
             label=r'DTFT $|X_1(\Omega)|$')
    plt.plot(muz / Nz * N, np.abs(X2z), 'C1',
             label=r'DTFT $|X_2(\Omega)|$')

    plt.xlim(0, N)
    plt.xlabel(r'DFT frequency index $\mu$')
    plt.ylabel(r'Magnitude (also absolute value)')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.stem(mu / N * 2 * np.pi, np.abs(X),
             basefmt='C0:', linefmt='C0:', markerfmt='C0o',
             label=r'DFT $|X_1[\mu]+X_2[\mu]|$')
    plt.plot(muz / Nz * 2 * np.pi, np.abs(Xz), 'C0',
             label=r'DTFT $|X_1(\Omega) + X_2(\Omega)|$')
    # due to linearity of convolution and Fourier transforms the same results
    # are obtained via:
    plt.stem(mu / N * 2 * np.pi, np.abs(X1+X2),
             basefmt='C7:', linefmt='C7:', markerfmt='C7.')
    plt.plot(muz / Nz * 2 * np.pi, np.abs(X1z+X2z), 'C7:')

    plt.xlim(0, 2*np.pi)
    plt.xlabel(r'DTFT frequency $\Omega$')
    plt.ylabel(r'Magnitude (also modulus)')
    plt.legend()
    plt.grid(True)

Then, we should define a certain (rather small) DFT length $N$. 

In [ ]:
# signal & DFT parameter
N = 2**4
k, mu = np.arange(N), np.arange(N)

Note that the scaling of DTFT is aligned to the DFT data for better visual cues.
We should clarify this even more in future.

In the upcoming examples, we always have two plots
- the upper graph over DFT frequency index $\mu$ shows the two magnitude DTFT spectra of $W(\Omega-\Omega_1)$, $W(\Omega-\Omega_2)$ and there corresponding DFT spectra (dots)
- the lower plots over DTFT frequency $\Omega$ shows the DTFT / DFT spectrum of the signal / spectrum superposition

Thus, in the upper plot we see how two Dirac impulses are smeared by the window spectrum. We can study the characteristics of
- the so called mainlobe (the shape around the spectrum's maximum) and
- the so called sidelobes (the shapes that are followed by the mainlobe)
- number of zeros (for optimum windows we have $N-1$ zeros in the DTFT spectrum). Note that $\Omega=0=2\pi$ is the same zero, due to periodicity.
- number of non-zeros coefficients in DFT

In the lower plot we see how the superposition of the two signals affects the resulting spectrum.
- Do we obtain a separate mainlobes for the two frequencies?
- Or do we obtain a rather broad single mainlobe, not able to tell that the signal is built from two frequencies?
- Do we have zeros in the spectrum?
- How about the sidelobe magnitude and the decay of the sidelobes?

# Rectangular Window

In [ ]:
# superposition of 2 complex exponential signals
mu1, mu2 = 6, 8  # we choose two DFT eigenfrequencies here
x1, x2 = 1*np.exp(1j * 2*np.pi/N * k*mu1), 1*np.exp(1j * 2*np.pi/N * k*mu2)
plot_dft_dtft()  # we see two separate main lobes
# the rectangular window has most narrow mainlobe width but at the price of
# highest sidelobes amplitude and slow(est?) sidelobe decay

In [ ]:
# superposition of 2 complex exponential signals
mu1, mu2 = 6, 7
x1, x2 = 1*np.exp(1j * 2*np.pi/N * k*mu1), 1*np.exp(1j * 2*np.pi/N * k*mu2)
plot_dft_dtft()  # we still see two separate main lobes
# mainlobes are now direct neighbours w.r.t. the DFT bins
# maximum of X1[mu] corresponds to a zero in X2[mu] and vice versa

# What happens if we put these signal frequencies even closer?
# Let's have a look at the next plot.

In [ ]:
# superposition of 2 complex exponential signals
mu1, mu2 = 6, 6.5
x1, x2 = 1*np.exp(1j * 2*np.pi/N * k*mu1), 1*np.exp(1j * 2*np.pi/N * k*mu2)
plot_dft_dtft()  # the two signal frequencies cannot longer clearly
# detected, since # their individual spectra smear into one single main lobe

# the orange spectrum (mu=6.5) is the worst case w.r.t. leakage effect, there
# all DFT coefficients have energy and thus suggest a multi-frequency
# signal. We know that this is only a single frequency signal, which was
# however very unfortunate cutted to fit in the chosen DFT length

# Kaiser-Bessel Window

In [ ]:
# signal & DFT parameter
N = 2**4
k, mu = np.arange(N), np.arange(N)

# window
w = kaiser(N, beta=3, sym=False)  # beta = 0 is rect window

# superposition of 2 complex exponential signals
# with non-rectangular windowing to reduce leakage effect
mu1, mu2 = 6, 8
x1, x2 = 1*w*np.exp(1j * 2*np.pi/N * k*mu1), 1*w*np.exp(1j * 2*np.pi/N * k*mu2)

plot_dft_dtft()
# with this window and window parametrization we are able to reduce the
# leakage effect, the two mainlobes are separated, but the valley between
# the two mainlobes has comparably large amplitude, it would be nice to have
# the two mainlobes better separated. We can do this by decreasing beta.
# Question: to what price?

# play around with beta = 0...3 and check the mainlobe width and sidelobe height
# ensure that beta=0 is identical with the rect window (this is a nice feature
# of the Kaiser-Bessel window)
# we can apply amplitude 0 to one of the signals just to see the spectrum
# of a single signal

In [ ]:
# signal & DFT parameter
N = 2**4
k, mu = np.arange(N), np.arange(N)

# window
w = kaiser(N, beta=7, sym=False)  # beta = 0 is rect window

# superposition of 2 complex exponential signals
# with non-rectangular windowing to reduce leakage effect
mu1, mu2 = 6, 8
x1, x2 = 1*w*np.exp(1j * 2*np.pi/N * k*mu1), 1*w*np.exp(1j * 2*np.pi/N * k*mu2)

plot_dft_dtft()
# with this window and window parametrization we are able to reduce the
# leakage effect even more, however now we broadend the two mainlobes in the
# upper plot
# such that they smear to one single mainlobe in the lower picture
# again this is always the general trade-off of windowing:
# broader mainlobe == smaller sidelobes but at a price of less frequency
# sensitivity 

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**Pole/Zeros Plots of Window Functions**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from scipy.signal import tf2zpk
from scipy.signal.windows import hann, hamming, chebwin, dpss, kaiser
from numpy.fft import fft, fftshift, ifft 

In [ ]:
def zplane_plot(z, p, k, title_str):
    """Plot pole/zero/gain plot of discrete-time, linear-time-invariant system.

    Note that the for-loop handling might be not very efficient
    for very long FIRs

    z...array of zeros in z-plane
    p...array of poles in z-zplane
    k...gain factor

    taken from own work
    URL = ('https://github.com/spatialaudio/signals-and-systems-exercises/'
           'blob/master/sig_sys_tools.py')

    currently we don't use the ax input parameter, we rather just plot
    in hope for getting an appropriate place for it from the calling function
    """
    # draw unit circle
    Nf = 2**7
    Om = np.arange(Nf) * 2*np.pi/Nf
    plt.plot(np.cos(Om), np.sin(Om), 'C7')

    try:  # TBD: check if this pole is compensated by a zero
        circle = Circle((0, 0), radius=np.max(np.abs(p)),
                        color='C7', alpha=0.15)
        plt.gcf().gca().add_artist(circle)
    except ValueError:
        print('no pole at all, ROC is whole z-plane')

    zu, zc = np.unique(z, return_counts=True)  # find and count unique zeros
    for zui, zci in zip(zu, zc):  # plot them individually
        plt.plot(np.real(zui), np.imag(zui), ms=7,
                 color='C0', marker='o', fillstyle='none')
        if zci > 1:  # if multiple zeros exist then indicate the count
            plt.text(np.real(zui), np.imag(zui), zci)

    pu, pc = np.unique(p, return_counts=True)  # find and count unique poles
    for pui, pci in zip(pu, pc):  # plot them individually
        plt.plot(np.real(pui), np.imag(pui), ms=7,
                 color='C3', marker='x')
        if pci > 1:  # if multiple poles exist then indicate the count
            plt.text(np.real(pui), np.imag(pui), pci)

    plt.text(0, +1, 'k=%f' % k)
    plt.text(0, -1, 'ROC for causal: white')
    plt.axis('square')
    # plt.axis([-2, 2, -2, 2])
    plt.xlabel(r'$\Re\{z\}$')
    plt.ylabel(r'$\Im\{z\}$')
    plt.title(title_str)
    plt.grid(True)
    
def freq_resp(w, label_str, Nz=2**10):
    Omega = 2*np.pi/Nz*np.arange(Nz)
    W = np.zeros(Nz)
    W[0:N] = w
    W = fftshift(fft(W))
    W /= np.max(np.abs(W))
    W = np.abs(W)
    W[W < 10**(-100/20)] = 10**(-100/20)  # avoid log10(0)
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twiny()
    ax1.plot(Omega/np.pi*180-180, 20*np.log10(W), label=label_str)
    ax1.set_xlabel(r'$\Omega$ / deg')
    ax1.set_ylabel(r'normalized $|W(\Omega)|$')
    ax1.set_xticks(np.angle(z)*180/np.pi)
    ax1.set_yticks(np.arange(-100, 10, 10))
    ax1.set_xlim(-180, 180)
    ax1.set_ylim(-100, 0)
    ax1.legend()
    ax1.grid(True, which='both')
    ax2.plot(Omega-np.pi, 20*np.log10(W))
    ax2.set_xlabel(r'$\Omega$ / rad')
    ax2.set_xlim(-np.pi, np.pi);

In [ ]:
N = 8  # length of window

In [ ]:
# don't change, this is needed below repeatedly:
k = np.arange(N)
a = np.zeros(N)
a[0] = 1

For a length $N$ window $w[k]$ with 
- $w[k]=0$ if $k<0$ and $k>N-1$

we have the z-transform of the window / finite length sequence

\begin{equation}
W(z) = \sum_{k=0}^{N-1} w[k] z^{-k} = w[0] + w[1] z^{-1} + w[2] z^{-2} + ... + w[N-1] z^{-(N-1)}
\end{equation}

We can rewrite 

\begin{equation}
W(z) = \frac{w[0] z^{N-1} + w[1] z^{N-2} + w[2] z^{N-3} + ... + w[N-1]}{z^{N-1}}
\end{equation}

- This rational function has $N-1$ poles in the origin, i.e. $z=0$
- Furthermore, it has $N-1$ zeros somewhere in the complex z-plane

For real valued $w[k]$ the zeros are either complex conjugate or real-valued (they lie on the real axis in the z-plane).

For the windowing process we are interested in the DTFT spectrum, which can be obtained when evaluating $W(z)$ along the unit circle, thus as

\begin{equation}
W(\Omega) = W(z)\big|_{z=\mathrm{e}^{\mathrm{j}\Omega}}
\end{equation}

Window design is about meaningful, if not to say clever, positioning of $N-1$ zeros, such that a desired trade-off
between **mainlobe width** and **sidelobe magnitude** / **sidelobe decay rate** is achieved.
Naturally, this design task has many, many solutions. Some of them are more meaningful than others, which is why they
were published as special window designs over the last decades.

For windowing we actually always aim at very low sidelobes. Thus all potential zeros of $W(z)$ should used as best as possible: putting them exactly onto the unit circle the impact is largest in the DTFT spectrum, since notches are realized. Windows that follow this design rule, are commonly termed optimum windows. The design constraints for the trade-off then tell us where exactly to put these zeros on the unit circle.

In the examples below we discuss very simple cases for rather small $N$.

**Example 1**

Although, is has worst leakage effect, the rectangular window is actually an optimum window. The design criterion is simply: provide the most narrow main lobe that is possible for $N$. The price you pay is the highest sidelobe magnitude and very slow sidelobe decay rate.

**Example 2**

This is a! symmetric Hann window (in older literature misnamed as Hanning window, don't do this anymore!). We see that two zeros are real valued and not on the unit circle. They don't shape the window spectrum in optimum manner.

Note that for didactical convenience we use a symmetric window here. For practical spectral analysis we rather should use the flag `sym=False` for scipy windows.

**Example 3**

Idea of Hamming window:
- taking the above Hann window and moving the two zeros onto the unit circle (they must be conjugate complex).
- position of zeros such that they put a notch at the two sidelobes of the Hann window left and right of the main lobe

Mainlobe shape remains, sidelobe magnitude is overall reduced compared to the Hann window.

## Rectangular Window

In [ ]:
w = np.ones(N)
[z, p, gain] = tf2zpk(w, a)
# zeros
print('zeros: angle in deg', np.angle(z)*180/np.pi)
print('zeros: abs', np.abs(z))
# zeros are equiangularly aligned on! the unit circle

In [ ]:
zplane_plot(z, p, gain, 'Rect')

In [ ]:
freq_resp(w, 'Rect')

## Symmetric Hann Window

In [ ]:
# a symmetric Hann window, two zeros are not used on! the unit circle
w = (1 - np.cos(2*np.pi/N*(k+1/2))) / 2
print('my hann:', w)
[z, p, gain] = tf2zpk(w, a)

In [ ]:
# Note that our Hann window above differs from the numpy/scipy definitions:
# our window does start and end with a non-zero coefficient, while numpy/scipy
# versions start/end with zeros effectively using two samples less for
# signal analysis/filter design
w_tmpn = np.hanning(N)  # numpy, note: 'hanning' is an old naming to be avoided
w_tmps = hann(N)  # scipy version
w_tmp = 0.5 - 0.5 * np.cos(2*np.pi/(N-1)*(k))  # manual version
print('numpy == scipy?', np.allclose(w_tmpn, w_tmps))
print('equal to manual?', np.allclose(w_tmpn, w_tmp))
print('numpy/scipy hann:', w_tmp)

In [ ]:
# zeros
print('zeros: angle in deg', np.angle(z)*180/np.pi)
print('zeros: abs', np.abs(z))

In [ ]:
zplane_plot(z, p, gain, 'Hann')

In [ ]:
freq_resp(w, 'Hann')

We have two zeros that are **not** on the unit circle. If we put them **onto** the unit circle their influence with regard the DTFT (i.e. the frequency response) can be made stronger. By that we can optimize the window, for example to attenuate a certain sidelobe. This idea is pursued by the Hamming window, which is shown next.

## Symmetric Hamming Window

In [ ]:
# a symmetric Hamming window
# we put notches into the two sidelobes 'left/right' of the main lobe
# of the above defined symmetric Hann window,
# thus here all zeros are used on! the unit circle
# which improves side lobe attenuation
w = 0.54 - 0.46 * np.cos(2*np.pi/N*(k+1/2))
print('my hamming:', w)
[z, p, gain] = tf2zpk(w, a)

In [ ]:
# Note that our Hamming window above differs from the numpy/scipy definitions
w_tmpn = np.hamming(N)
w_tmps = hamming(N)
w_tmp = 0.54 - 0.46 * np.cos(2*np.pi/(N-1)*(k))
print('numpy == scipy?', np.allclose(w_tmpn, w_tmps))
print('equal to manual?', np.allclose(w_tmpn, w_tmp))
print('numpy/scipy hamming:', w_tmp)

In [ ]:
# +-112.03298762 is approx +-(90 + 135) / 2
# this is the angle of the first side lobe
print('zeros-> angle in deg:', np.angle(z)*180/np.pi)
# so put a zero in the middle of the 'first/last' two zeros
# now all zeros are on! the unit circle, leading to an optimum design

# zeros
print('zeros: angle in deg', np.angle(z)*180/np.pi)
print('zeros: abs', np.abs(z))
# all zeros on! the unit circle (since all np.abs(z) == 1) 

In [ ]:
zplane_plot(z, p, gain, 'Hamming')

In [ ]:
freq_resp(w, 'Hamming')

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**Parametric Windows: Dolph-Chebyshev, Slepian and Kaiser-Bessel Window**,
Winter Semester 2021/22 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Parametric Windows: Dolph-Chebyshev, Slepian, Kaiser-Bessel Window

There are numerous window types, all developed for special requirements.
In the initial [DFT & Windowing](dft_windowing.ipynb) exercise and the detailed [DFT tutorial](dft_windowing_tutorial/dft_windowing_tutorial.pdf) we have learned about two very simple computable and very often used **non-parametric** windows, the Hann and the Hamming window.
Non-parametric means that by desired window length $M$, the window and its DTFT spectrum are fully determined.
In other words, if another spectral characteristics - very often we need higher spectral resolution - is asked for, then the only variable to change is $M$.

The **Hann** window is **not optimal**, since it does not use two of its potential zeros to shape the sidelobes of its DTFT spectrum.
The Hamming window introduces two additional zeros to the Hann window to reduce the level of the first sidelobe.
Note that the Hann window is still often used nowadays, not due to its non-optimum spectrum, but rather due to its simple calculation of the window signal $w[k]$ requiring only a cosine and weight of $1/2$.

So called **parametric** windows have additional parameters, that allow to meet certain constraints for a given overall design criterion.
Two of the most prominent - in fact with these we probably can manage the majority of windowing applications - are the **Dolph-Chebyshev** and the **Kaiser-Bessel** window.
These are optimum window designs.
The Kaiser-Bessel window itself can be considered as an approximation of the so called **discrete prolate spheroidal sequences** (DPSS, aka Slepian) window.

We will discuss these windows below in terms of their design criteria and the resulting additional parameter that can be set up to meet a desired constraint.
TBD...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from numpy.fft import fft, fftshift, ifft 
#from scipy.fft import fft, fftshift, ifft
from scipy.signal.windows import chebwin, kaiser, dpss
from scipy.signal import tf2zpk

## Dolph-Chebyshev Window

In [ ]:
# parameter for the window
M = 64  # window length
SidelobeAttenuation = 50  # in dB

In [ ]:
w = chebwin(M, at=SidelobeAttenuation, sym=True)
# note on the code from:
# https://github.com/scipy/scipy/blob/v0.19.0/scipy/signal/windows.py#L1293-L1416
#
# The tricky part here is that the analytic equations given in most textbooks
# such as e.g. R.G. Lyons (2011): "Understanding Digital Signal Processing",
# Prentice Hall, Upper Saddle River, 3rd ed., eq. (5.17)
# cannot be straightforwardly implemented due to numerical issues
# however, certain parts of the equation can be identified as Chebyshev polynomials
# (the window has its name from there) for which relations can be utilized.
# This is done in the scipy code for p[x < -1], p[np.abs(x) <= 1]

In [ ]:
plt.plot(w, 'C0o-', ms=5)
plt.xlabel(r'sample $k$')
plt.ylabel(r'$w[k]$')
plt.title('Dolph-Chebyshev window')
plt.grid(True)

In [ ]:
Nz = 2**16  # zeropadding of window -> quasi-cont resolution of W for DTFT
Omega = 2*np.pi/Nz * np.arange(Nz)
wz = np.zeros(Nz)
wz[0:M] = w
Wz = fft(wz)
Wzmax = np.max(20*np.log10(np.abs(Wz)))
plt.plot(Omega, 20*np.log10(np.abs(Wz))-Wzmax, 'k')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(fftshift(Wz)))-Wzmax, 'C0')
plt.plot([np.pi, np.pi], [-120, 10], 'C7')
plt.plot([-np.pi, 2*np.pi], [-50, -50], 'dimgray')
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'$20 \log_{10}|W(\Omega)|$ in dB')
plt.xlim(-np.pi, 2*np.pi)
plt.ylim(-100, 10)
plt.yticks(np.arange(-100,10,10))
plt.title('DTFT Spectrum of Dolph-Chebyshev Window with %d Zeros' %(M-1))
plt.grid(True)

We might figure out the design criterion of the Dolph-Chebyshev by ourselves, when inspecting the DTFT spectrum of it.
Hint: How is the additional parameter linked to the DTFT spectrum.
Vary `M` and `SidelobeAttenuation` and check for changes.
For small `M` (to make analysis convenient) check how the zeros are placed in the spectrum to meet the design criterion.

## DPSS Window aka Slepian Window

The design criterion of the **Slepian** (also named **discrete prolate spheroidal sequences** or **digital prolate spheroidal sequences** (DPSS)) window is **maximum energy** concentration in the **main lobe** for a given mainlobe **bandwidth**.
Actually, this is what we typically ask for signal analysis, if no other specific constraints about the sidelobes positions and there levels are requested.

Recall, that the DTFT spectrum for the 'ideal world' window is the Dirac impulse (steming from the practically not feasible infinite rectangular window), so mainlobe energy concentration seems to be a very good approach to get close to it.

See

- Surendra Prasad (1982): "On an Index for Array Optimization and the Discrete Prolate Spheroidal Functions." In:
IEEE Transactions on Antennas and Propagation, vol. AP-30, no. 5, pg. 1021-1023, [DOI: 10.1109/TAP.1982.1142900](https://doi.org/10.1109/TAP.1982.1142900)

- Michael Möser (1988): "Analyse und Synthese akustischer Spektren.", Springer, Berlin, Kap. 3.2.2, [DOI: 10.1007/978-3-642-93374-5](https://doi.org/10.1007/978-3-642-93374-5)

- Julius O. Smith (2011): Spectral Audio Signal Processing, online lecture of CCRMA, Stanford University, https://ccrma.stanford.edu/~jos/sasp/Slepian_DPSS_Window.html


for treatments how to derive the Slepian window.

Challenging question:
What window type results if we ask the Slepian window to produce mainlobe bandwidth $\rightarrow 0$?
Implement a test case to approach the answer. 

In [ ]:
# parameter for the window
M = 64  # window length
bw = 2*np.pi/45  # -3dB bandwidth of the main lobe in terms of digital frequency
# empirically found for the specific window length
NW = bw * M/4

For the chosen example, this Slepian window has (i) the same window length and (ii) the first sidelobe is at about -50 dB like the Dolph-Chebyshev window above.

In [ ]:
w = dpss(M, NW, sym=True)  # https://docs.scipy.org/doc/scipy-1.7.1/reference/generated/scipy.signal.windows.dpss.html
plt.plot(w, 'C0o-', ms=5)
plt.xlabel(r'sample $k$')
plt.ylabel(r'$w[k]$')
plt.title('Slepian window')
plt.grid(True)

In [ ]:
Nz = 2**16  # zeropadding of window -> quasi-cont resolution of W for DTFT
Omega = 2*np.pi/Nz*np.arange(Nz)
wz = np.zeros(Nz)
wz[0:M] = w
Wz = fft(wz)
Wzmax = np.max(20*np.log10(np.abs(Wz)))
plt.plot([np.pi, np.pi], [-120, 10], 'k')
plt.plot([-np.pi, 2*np.pi], [-3.01, -3.01], 'C1')
plt.plot([-np.pi, 2*np.pi], [-50, -50], 'dimgray')
plt.plot([-bw/2, -bw/2], [-120, 10], color='C1')
plt.plot([+bw/2, +bw/2], [-120, 10], color='C1')
plt.plot(Omega, 20*np.log10(np.abs(Wz))-Wzmax, 'k')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(fftshift(Wz)))-Wzmax, 'C0')
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'$20 \log_{10}|W(\Omega)|$ in dB')
plt.xlim(-np.pi, 2*np.pi)

# zoom the main lobe
if False:
    plt.xlim(-5*bw, +5*bw)

plt.ylim(-100, 10)
plt.yticks(np.arange(-100,10,10))
plt.title('DTFT Spectrum of DPSS Window aka Slepian Window with %d Zeros' % (M-1))
plt.grid(True)

## Kaiser-Bessel Window

The Kaiser-Bessel window is an **approximation** of the Slepian window **for large window lengths** $M$, note however that they will be **never identical**.
In the days of its **invention by Kaiser** it was much easier to compute it than the discrete prolate spheroidal sequences discussed above. 
This is due to the explicit given equation for the Kaiser-Bessel window, whereas for the Slepian window an eigenwert problem for a $M/2$ matrix has to be numerically solved.
The Kaiser-Bessel window requires the [zeroth-order modified **Bessel** function of the first kind](https://dlmf.nist.gov/10.25) $I_0(\cdot)$ to calculate the window signal $w[k]$.
Thus, the given name in the DSP literature.

TBD...

For the upcoming discussion we need pole/zeros plot. So let's define a convenient plotting routine before:

In [ ]:
def zplane_plot(z, p, k):
    """Plot pole/zero/gain plot of discrete-time, linear-time-invariant system.

    Note that the for-loop handling might be not very efficient
    for very long FIRs

    z...array of zeros in z-plane
    p...array of poles in z-zplane
    k...gain factor

    taken from own work
    URL = ('https://github.com/spatialaudio/signals-and-systems-exercises/'
           'blob/master/sig_sys_tools.py')

    currently we don't use the ax input parameter, we rather just plot
    in hope for getting an appropriate place for it from the calling function
    """
    # draw unit circle
    Nf = 2**7
    Om = np.arange(Nf) * 2*np.pi/Nf
    plt.plot(np.cos(Om), np.sin(Om), 'C7')

    try:  # TBD: check if this pole is compensated by a zero
        circle = Circle((0, 0), radius=np.max(np.abs(p)),
                        color='C7', alpha=0.15)
        plt.gcf().gca().add_artist(circle)
    except ValueError:
        print('no pole at all, ROC is whole z-plane')

    zu, zc = np.unique(z, return_counts=True)  # find and count unique zeros
    for zui, zci in zip(zu, zc):  # plot them individually
        plt.plot(np.real(zui), np.imag(zui), ms=7,
                 color='C0', marker='o', fillstyle='none')
        if zci > 1:  # if multiple zeros exist then indicate the count
            plt.text(np.real(zui), np.imag(zui), zci)

    pu, pc = np.unique(p, return_counts=True)  # find and count unique poles
    for pui, pci in zip(pu, pc):  # plot them individually
        plt.plot(np.real(pui), np.imag(pui), ms=7,
                 color='C3', marker='x')
        if pci > 1:  # if multiple poles exist then indicate the count
            plt.text(np.real(pui), np.imag(pui), pci)

    plt.text(0, +1, 'k=%f' % k)
    plt.text(0, -1, 'ROC for causal: white')
    plt.axis('square')
    # plt.axis([-2, 2, -2, 2])
    plt.xlabel(r'$\Re\{z\}$')
    plt.ylabel(r'$\Im\{z\}$')
    plt.grid(True)

## Comparison of the Windows

What are the differences between the discussed **parametric** window types of same length and about the same level of the first sidelobe?

In [ ]:
# check for increasing integer L
# how Slepian and the Kaiser-Bessel approximation converge, but never are exactly the same
L = 1

M = np.int(2**6 * L)
Nz = 2**6 * M  # zeropadding of window -> quasi-cont resolution of W for DTFT
k = np.arange(M)
bw = 2*np.pi/(45*L)  # for dpss
NW = bw * M/4  # for dpss
beta = 6.85  # for kaiser
Omega = 2*np.pi/Nz * np.arange(Nz)
wdc = chebwin(M, at=50, sym=True)  # dolph-chebyshev
wslep = dpss(M, NW, sym=True)  # slepian
wkb = kaiser(M, beta, sym=True)  # kaiser-bessel

Wdc = np.zeros(Nz)
Wdc[0:M] = wdc
Wdc = fftshift(fft(Wdc))
Wdc /= np.max(np.abs(Wdc))

Wslep = np.zeros(Nz)
Wslep[0:M] = wslep
Wslep = fftshift(fft(Wslep))
Wslep /= np.max(np.abs(Wslep))

Wkb = np.zeros(Nz)
Wkb[0:M] = wkb
Wkb = fftshift(fft(Wkb))
Wkb /= np.max(np.abs(Wkb))

plt.figure(figsize=(7, 7))
plt.subplot(2, 1, 1)
plt.plot(k, wdc, label='Dolph-Chebyshev')
plt.plot(k, wslep, label='DPSS aka Slepian')
plt.plot(k, wkb, label='Kaiser-Bessel', color='C3')
plt.xlabel(r'sample $k$')
plt.ylabel(r'$w[k]$')
plt.title('M=%d' % M)
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot((-np.pi, +np.pi), (-50, -50), color='dimgray')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wdc)))
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wslep)))
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wkb)), color='C3')
plt.ylim(-100, 0)
plt.xlim(-np.pi, +np.pi)

# zoom the main lobe:
if False:
    plt.xlim(-3*bw, +3*bw)

plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'$20 \log_{10}|W(\Omega)|$ in dB')
plt.grid(True)

In [ ]:
a = np.zeros(M)
a[0] = 1

In [ ]:
[z, p, gain] = tf2zpk(wdc, a)
zplane_plot(z, p, gain)
plt.title('Dolph-Chebyshev')
np.abs(z)  # all zeros on! the unit-circle == optimum design

In [ ]:
[z, p, gain] = tf2zpk(wslep, a)
zplane_plot(z, p, gain)
plt.title('DPSS aka Slepian')
np.abs(z)  # all zeros on! the unit-circle == optimum design

In [ ]:
[z, p, gain] = tf2zpk(wkb, a)
zplane_plot(z, p, gain)
plt.title('Kaiser-Bessel')
np.abs(z)  # all zeros on! the unit-circle == optimum design

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises